Implmenting section 4 "Hiding the Internal State of the Algorithm" in:

Golle P. (2006) A Private Stable Matching Algorithm. In: Di Crescenzo G., Rubin A. (eds) Financial Cryptography and Data Security. FC 2006. Lecture Notes in Computer Science, vol 4107. Springer, Berlin, Heidelberg. https://doi-org.proxygw.wrlc.org/10.1007/11889663_5

Let $A_1...A_3$ denote 3 men and $B_1...B_3$ denote three women.

Preferences:

$A_1$: $a_1=(0,2,1)$

$A_2$: $a_2=(1,2,0)$

$A_3$: $a_3=(2,1,0)$

---

$B_1$: $b_1=(1,2,0)$

$B_2$: $b_2=(0,2,1)$

$B_3$: $b_3=(0,1,2)$

In [6]:
a_1 = [0, 2, 1]
a_2 = [1, 2, 0]
a_3 = [2, 1, 0]
men_preferences = [a_1, a_2, a_3]

b_1 = [1, 2, 0]
b_2 = [0, 2, 1]
b_3 = [0, 1, 2]

In [2]:
def hidden_internal_state(a_prefs, b_prefs):
    # Adding "fake" men
    